In [2]:
import pandas as pd
import openpyxl
import math
import statistics

# Repartiment d'escons per provincia segons el mètode

Diccionari amb el nombre de votants per provincia

In [7]:
votants = pd.read_excel("C:/Users/Pau Program/Documents/Mirror/UIB/MUSI/TFM/Votants_per_provincia.xlsx")

provincies = list(votants.iloc[:,0])
escons_provincia = list(votants.iloc[:,6])

escons_per_provincia = {provincia:escons for provincia, escons in zip(provincies, escons_provincia)} 

{'A Coruña': 8.0,
 'Alacant': 12.0,
 'Albacete': 4.0,
 'Almería': 6.0,
 'Álava': 4.0,
 'Asturias': 7.0,
 'Ávila': 3.0,
 'Badajoz': 5.0,
 'Barcelona': 32.0,
 'Bizkaia': 8.0,
 'Burgos': 4.0,
 'Cáceres': 4.0,
 'Cádiz': 9.0,
 'Cantabria': 5.0,
 'Castelló': 5.0,
 'Ceuta': 1.0,
 'Ciudad Real': 5.0,
 'Córdoba': 6.0,
 'Cuenca': 3.0,
 'Gipuzkoa': 6.0,
 'Girona': 6.0,
 'Granada': 7.0,
 'Guadalajara': 3.0,
 'Huelva': 5.0,
 'Huesca': 3.0,
 'Illes Balears': 8.0,
 'Jaén': 5.0,
 'La Rioja': 4.0,
 'Las Palmas': 8.0,
 'León': 4.0,
 'Lleida': 4.0,
 'Lugo': 4.0,
 'Madrid': 37.0,
 'Málaga': 11.0,
 'Melilla': 1.0,
 'Murcia': 10.0,
 'Navarra': 5.0,
 'Ourense': 4.0,
 'Palencia': 3.0,
 'Pontevedra': 7.0,
 'Salamanca': 4.0,
 'Santa Cruz de Tenerife': 7.0,
 'Segovia': 3.0,
 'Sevilla': 12.0,
 'Soria': 2.0,
 'Tarragona': 6.0,
 'Teruel': 3.0,
 'Toledo': 6.0,
 'Valencia': 16.0,
 'Valladolid': 5.0,
 'Zamora': 3.0,
 'Zaragoza': 7.0,
 nan: nan}

A Espanya, el repartiment d'escons per provincia ve establert en base a la quota Hare com s'indica a (insertar referencia de la junta electoral)

El mètode D'Hont estableix que un partit pot obtenir un escon a la seva circumscirpció provincial només si supera el 3% dels vots totals. Així, els vots en blanc a l'únic que afecten és a incrementar el nombre de vots necessaris per obtenir un escon.

## Funcions

In [3]:
def quota(p, h):
    P = sum(p)
    q = [h*pi/P for pi in p]

    return q

def hamilton(p, h):
    # Calculam les quotes i els floors de les mateixes
    q = quota(p, h)
    b = [math.floor(qi) for qi in q]
    
    # Extreim la part decimal de les quotes
    dec = [qi-bi for (qi, bi) in zip(q, b)]

    # Guardam la posició de cada quota per saber a quina població correspon al realitzar l'ordenació
    dec_pos = [(d,i) for (d,i) in zip(dec, range(0, len(p)))]
    ordenada = sorted(dec_pos, key=lambda x: x[0], reverse = True)

    # Calculam els escons que queden a repartir
    rest = h -sum(b)

    # Afegim un escon a les h-sum(b) poblacions amb les quotes amb part decimal més gran
    for i in range(0, rest):
        b[ordenada[i][1]] += 1
        
    return b

In [4]:
def divisor_method(population, house, d):
    escons = [0]*len(population)
    if d == d_adams or d == d_dean or d == d_hill:
        for i in range(0, house):
            if i < len(escons):
                escons[i] += 1 

    while sum(escons) < house:
        populations_compare = [p/d(e) for p,e in zip(population,escons)]
        max_val = max(populations_compare)
        max_ind = populations_compare.index(max_val)

        escons[max_ind] += 1
    
    return escons

def d_adams(a):
    return a

def d_jefferson(a):
    return a + 1

def d_webster(a):
    return a + 1/2

def d_dean(a):
    return (a*(a+1))/(a + 1/2)

def d_hill(a):
    return math.sqrt(a*(a+1))

In [5]:
def divisor(p, h, d):
    # El primer divisor a provar és ds
    ds = sum(p)/h
    divisor = ds
    inf = 0
    sup = ds**10
    a = [0]*len(p)
    
    i = 0
    while sum(a) != h and i <= 1000:
        i += 1
        # Calculam les quotes modificades
        q = [pi/divisor for pi in p]
        # Calculam el repartiment
        a = [math.floor(qi) if (d(math.floor(qi)) <= qi and qi <= d(math.floor(qi)+1)) else math.ceil(qi) for qi in q]
        # Si la suma dels escons repartits és menor que la mida de la cambra, agafam un divisor menor
        if sum(a) < h:
            sup = divisor
            divisor = statistics.mean([inf,sup])
        # Si la suma dels escons a repartir és superior que la mida de la cambra, agafam un divisor superior
        if sum(a) > h:
            inf = divisor
            divisor = statistics.mean([inf,sup])

    # Si el mètode no ha estat capaç de trobar el divisor aplicam el mètode descrit a divisor_method
    if sum(a) != h:
        a = divisor_method(p,h,d)                
    return a

In [6]:
def Dhont(population, house):
    escons = [0]*len(population)
    while sum(escons) < house:
        populations_compare = [p/(e+1) for p,e in zip(population,escons)]

        max_val = max(populations_compare)
        max_ind = populations_compare.index(max_val)

        escons[max_ind] += 1
    
    return escons

Jefferson y D'Hont son exactament el mateix mètode.

In [7]:
p = [287997, 188184, 81345, 66996, 33606]
h = 4

print(Dhont(p, h))
print(hamilton(p,h))
print(divisor(p,h,d_adams))
print(divisor_method(p,h,d_webster))
print(divisor(p,h,d_dean))
print(divisor(p,h,d_hill))

[3, 1, 0, 0, 0]
[2, 1, 1, 0, 0]
[3, 1, 0, 0, 0]
[2, 1, 1, 0, 0]
[3, 1, 0, 0, 0]
[3, 1, 0, 0, 0]


### Càlcul d'escons pels diferents mètodes

In [8]:
# Llegim l'excel on tenim l'informació de les provincies
vots_provincia = 'C:/Users/Pau Program/Documents/Mirror/UIB/MUSI/TFM/VOTS_PER_PROVINCIA_mes_3_percent.xlsx'

# Carregar l'arxiu excel
vots = pd.ExcelFile(vots_provincia)

# Nom de les fulles
noms_fulles = vots.sheet_names

vots_provincia_dhont = 'C:/Users/Pau Program/Documents/Mirror/UIB/MUSI/TFM/VOTS_PER_PROVINCIA_dhont.xlsx'


with pd.ExcelWriter(vots_provincia_dhont, engine='openpyxl') as writer:
    # Iterar sobre cada hoja
    for fulla in noms_fulles:
        # llegim la fulla
        df = vots.parse(fulla)
        print(fulla)

        # separam el nombre de vots a una llista
        p = list(df.iloc[:, 1])
        # veim el nombre d'escons que corresponen a la provincia
        h = escons_per_provincia[fulla]

        # calculam el nombre d'escons que correspon a cada partit (suponiendo que tienes la función Dhont definida)
        q = quota(p, h)
        escons_Dhont = Dhont(p, h)
        escons_hamilton = hamilton(p,h)
        escons_adams = divisor(p,h,d_adams)
        escons_webster = divisor_method(p,h,d_webster)
        escons_dean = divisor(p,h,d_dean)
        escons_hill = divisor(p,h,d_hill)

        df.loc[:, "QUOTA"] = q
        df.loc[:, "ESCONS D'HONT"] = escons_Dhont
        df.loc[:,"ESCONS HAMILTON"] = escons_hamilton
        df.loc[:,"ESCONS ADAMS"] = escons_adams
        df.loc[:,"ESCONS WEBSTER"] = escons_webster
        df.loc[:,"ESCONS DEAN"] = escons_dean
        df.loc[:,"ESCONS HILL"] = escons_hill

        # Escribir el DataFrame en una nueva hoja con el nombre de la fulla actual
        df.to_excel(writer, sheet_name=fulla, index=False)

A Coruña
Alacant
Albacete
Almería
Álava
Asturias
Ávila
Badajoz
Barcelona
Bizkaia
Burgos
Cáceres
Cádiz
Cantabria
Castelló
Ceuta
Ciudad Real
Córdoba
Cuenca
Gipuzkoa
Girona
Granada
Guadalajara
Huelva
Huesca
Illes Balears
Jaén
La Rioja
Las Palmas
León
Lleida
Lugo
Madrid
Málaga
Melilla
Murcia
Navarra
Ourense
Palencia
Pontevedra
Salamanca
Santa Cruz de Tenerife
Segovia
Sevilla
Soria
Tarragona
Teruel
Toledo
Valencia
Valladolid
Zamora
Zaragoza


In [9]:
print(noms_fulles)

['A Coruña', 'Alacant', 'Albacete', 'Almería', 'Álava', 'Asturias', 'Ávila', 'Badajoz', 'Barcelona', 'Bizkaia', 'Burgos', 'Cáceres', 'Cádiz', 'Cantabria', 'Castelló', 'Ceuta', 'Ciudad Real', 'Córdoba', 'Cuenca', 'Gipuzkoa', 'Girona', 'Granada', 'Guadalajara', 'Huelva', 'Huesca', 'Illes Balears', 'Jaén', 'La Rioja', 'Las Palmas', 'León', 'Lleida', 'Lugo', 'Madrid', 'Málaga', 'Melilla', 'Murcia', 'Navarra', 'Ourense', 'Palencia', 'Pontevedra', 'Salamanca', 'Santa Cruz de Tenerife', 'Segovia', 'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo', 'Valencia', 'Valladolid', 'Zamora', 'Zaragoza']


In [9]:
partits = []

for fulla in noms_fulles:
    df = vots.parse(fulla)
        
    # separam el nombre de vots a una llista
    p = list(df.iloc[:, 0])
    partits += p

print(set(partits))


{'CCa', 'PNV', 'SUMAR', 'NC-bc', 'JxCAT', 'CpM', 'EXISTE', 'B.N.G.', 'U.P.L.', 'EH Bildu', 'PP', 'U.P.N.', 'XAV', 'SY', 'PSOE', 'CUP', 'VOX', 'ERC'}
